In [1]:
import importlib
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
importlib.reload(network_tools)
importlib.reload(firewall_tools)

<module 'tools.firewall_tools' from '/home/federico/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/firewall_tools.py'>

In [2]:
alerts = await network_tools.get_alerts(time_window=5)
len(alerts.get('security_events', []).get('alerts'))

29

In [4]:
alerts = await network_tools.get_alerts(time_window=100)
alerts

{'security_events': {'alerts': [{'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.10',
    'dest_port': 3306,
    'flow_id': 1015776059242663,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '192.168.100.2',
    'src_port': 56370,
    'timestamp': '2025-10-11T18:03:07.564183+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.4',
    'dest_port': 587,
    'flow_id': 882706194041258,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '192.168.100.2',
    'src_port': 56370,
    'timestamp': '2025-10-11T18:03:07.664273+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.4',
    'dest_port': 636,
    'flow_id': 893392437910766,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '192.168.100.2',
    'src_por

In [4]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': '09bde572be3a',
   'name': 'xdebug-rce-xdebug2-1',
   'image': 'vulhub/php:7.1-xdebug',
   'status': 'running',
   'created': '2025-10-10T16:41:09.883302905Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'},
  {'id': 'cb92ecc803cd',
   'name': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'status': 'running',
   'created': '2025-10-10T15:56:22.97521297Z',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip_address': '172.20.0.2'},
  {'id': '2d9386380438',
   'name': 'cve-2024-36401-web-1',
   'image': 'vulhub/geoserver:2.23.2',
   'status': 'running',
   'created': '2025-10-10T15:38:14.023288903Z',
   'ports': ['5005/tcp', '8080/tcp'],
   'ip_address': '172.20.0.8'},
  {'id': '480b4c7e15db',
   'name': 'cve-2013-4547-nginx-1',
   'image': 'vulhub/nginx:1.4.2',
   'status': 'running',
   'created': '2025-10-10T15:38:13.732647967Z',
   'ports': ['443/tcp', '80/tcp'],
   'ip_address': '172.20.0.30'},
  {'id': 'f42411294fe9',
   

In [6]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     tcp  --  172.20.0.8           192.168.100.0/24    
2    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.8          
3    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
4    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
5    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
6    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
7    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
8    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [7]:
await firewall_tools.remove_firewall_rule([1, 2])

INFO:tools.firewall_tools:Removing firewall rules: [1, 2]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  172.20.0.8           192.168.100.0/24',
  'REMOVED rule #2:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.8']}

In [7]:
honeypot_ip = '172.20.0.7' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip, port=8080)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.7 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.7 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.7 -> 192.168.100.2/24 (tcp)'}